# Soil classification based on images using CNN

In [1]:
#importing the essentials libraries 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator  #for imagedata generation
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix


In [2]:
#Organizing the dataset into two directories: one for training images and one for testing images.
#Each directory  have subdirectories for each class (e.g., red soil, black soil).

train_data_gen = ImageDataGenerator(         # to preprocess and augment data (e.g., scaling, flipping, rotation).
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_data_gen = ImageDataGenerator(rescale=1.0 / 255)


In [3]:
train_generator =  train_data_gen.flow_from_directory('C:\\Users\\HP\\OneDrive\\Desktop\\Dataset\\Train'    #path of train data 


    ,
    target_size=(224, 224),  #adjusting the image size
    batch_size=32,             
    class_mode='categorical'
)

test_generator = test_data_gen.flow_from_directory('C:\\Users\\HP\\OneDrive\\Desktop\\Dataset\\test'   #path of test data 
    ,
    target_size=(224, 224),   #adjusting the image size
    batch_size=32,
    class_mode='categorical'
)


Found 1214 images belonging to 4 classes.
Found 339 images belonging to 4 classes.


In [4]:
#define the model
model = Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)), #convolution layer + activation function
    layers.MaxPooling2D(2, 2),                                               #pooling layer
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),                                                           #Flatten layer
    layers.Dense(128, activation='relu'),
    layers.Dense(4, activation='softmax')
])


In [5]:
#using adam optimizer 
#compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
#training the model
model.fit(train_generator, epochs=10, validation_data=test_generator)


Epoch 1/10
38/38 [==============================] - 107s 3s/step - loss: 1.1850 - accuracy: 0.5832 - val_loss: 0.6722 - val_accuracy: 0.7198
Epoch 2/10
38/38 [==============================] - 83s 2s/step - loss: 0.4983 - accuracy: 0.8031 - val_loss: 0.4446 - val_accuracy: 0.8142
Epoch 3/10
38/38 [==============================] - 84s 2s/step - loss: 0.4694 - accuracy: 0.7932 - val_loss: 0.3831 - val_accuracy: 0.8112
Epoch 4/10
 7/38 [====>.........................] - ETA: 1:02 - loss: 0.4621 - accuracy: 0.7857

In [ ]:
#evaluation of the model
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test accuracy: {test_accuracy}")


In [ ]:
# Save the trained model in the TensorFlow SavedModel format
tf.saved_model.save(model, 'soil_classification_model')



In [ ]:
# Load the SavedModel using tf.saved_model.load
loaded_model = tf.saved_model.load('soil_classification_model')


In [ ]:
# Save the trained model in the native Keras format
model.save('soil_classification_model.keras')


In [ ]:
import tensorflow as tf

# Load the trained model
model = tf.keras.models.load_model('soil_classification_model.keras')


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# constants for testing
BATCH_SIZE = 32 
IMAGE_SIZE = (224, 224)  #  match the image size used during training

# data generator for the testing data
test_data_gen = ImageDataGenerator(rescale=1.0 / 255)

test_generator = test_data_gen.flow_from_directory(
    'C:\\Users\\HP\\OneDrive\\Desktop\\Dataset\\test',  # path to testing data
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False  # to ensure predictions match filenames
)

# predictions on the testing data
predictions = model.predict(test_generator)


# Convert class probabilities to class labels
predicted_classes = predictions.argmax(axis=1)
class_names = list(test_generator.class_indices.keys())  # Get class names
for i, filename in enumerate(test_generator.filenames):
    print(f"File: {filename}, Predicted Class: {class_names[predicted_classes[i]]}")


In [ ]:
true_labels = test_generator.classes

#  accuracy
accuracy = accuracy_score(true_labels, predicted_classes)

#  F1 score
f1 = f1_score(true_labels, predicted_classes, average='weighted')

# classification report with precision, recall, and F1 score for each class
report = classification_report(true_labels, predicted_classes, target_names=class_names)

# Create a confusion matrix
confusion = confusion_matrix(true_labels, predicted_classes)

print(f"Test Accuracy: {accuracy}")
print(f"F1 Score (weighted): {f1}")
print("Confusion Matrix:")
print(confusion)
print("Classification Report:")
print(report)


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np  # Import NumPy

# Load the trained model
model = tf.keras.models.load_model('soil_classification_model.keras')

# Load and preprocess a new image
new_image = load_img(r"C:\Users\HP\Downloads\as.jpeg", target_size=(224, 224))
new_image = img_to_array(new_image)
new_image = new_image / 255.0  # Normalize the pixel values

# Expand dimensions to match the input shape of the model 
new_image = new_image[np.newaxis, ...]  

#  prediction
predictions = model.predict(new_image)

# Convert class probabilities to class label
predicted_class = predictions.argmax()

class_names = ['Alluvial soil ', '   Black Soil ', 'Clay soil', ' Red soil']    # Define the soil type names

predicted_soil_type = class_names[predicted_class]         # Print the predicted soil type
print(f"Predicted Soil Type: {predicted_soil_type}")